In [ ]:
'''
This notebook constitutes and ETL for IMDB data.
It downloads and prepares the data from IMDB (https://www.imdb.com/interfaces/)
such that the data files can be consumed by the Streamlit app. 
Roughly, the steps carried out are the following:
-Download and unpack zipped files from IMDB with data on movie titles/ratings and the people involved
-Combine title basics data with ratings
-Reduce data sizes by label encoding title IDs and subsetting data
-Generate a list of similar movies per movie (computationally heavy!) if required (or load already processed file with similar movies)
-Create mapping table for genres for faster subsetting of movies based on genre (combinations) within app
-Store processed data and delete downloads from IMDB
'''

In [ ]:
import pandas as pd
import pickle
import requests
import numpy as np
from datetime import datetime
import swifter
from sklearn.feature_extraction.text import CountVectorizer 
from sklearn.preprocessing import LabelEncoder

In [ ]:
# download zipped files from imdb and store as pickle files
# see https://www.imdb.com/interfaces/ for more information on the individual table (basics/ratings/principals)
title_basics_file_prefix     = 'title.basics'
name_basics_file_prefix      = 'name.basics'
title_ratings_file_prefix    = 'title.ratings'
title_principals_file_prefix = 'title.principals'
files_prefixes = [title_ratings_file_prefix, name_basics_file_prefix, title_basics_file_prefix,title_principals_file_prefix]
file_names     = [file_prefix + '.tsv.gz' for file_prefix in files_prefixes]

for file_prefix in files_prefixes:
    filename  = file_prefix + '.tsv.gz'
    url = 'https://datasets.imdbws.com/' + filename
    with open(filename, "wb") as f:
        r = requests.get(url)
        f.write(r.content)
        
# save all tables one by one into separate sav files
for file in file_names:
    pickle.dump(pd.read_table(file,sep="\t",low_memory=False, na_values=["\\N","nan"]),
                open(file[:-4]+".sav","wb"))

In [ ]:
# combine title basics and ratings per movie 

# TITLE BASICS TABLE #
df_title_basics = pickle.load(open(title_basics_file_prefix + '.ts.sav',"rb"))
df_title_basics = df_title_basics[df_title_basics.isAdult == 0]
df_title_basics = df_title_basics[(df_title_basics.titleType == "movie") | (df_title_basics.titleType == "tvMovie")]
df_title_basics.drop(["isAdult","endYear",'titleType','originalTitle'],axis=1,inplace=True)

# TITLE RATINGS TABLE #
df_ratings = pickle.load(open(title_ratings_file_prefix + '.ts.sav',"rb")) # tconst	averageRating	numVotes

title_basics_ratings = df_title_basics.merge(df_ratings, on = 'tconst')
title_basics_ratings = \
title_basics_ratings[title_basics_ratings.numVotes > \
                     title_basics_ratings.groupby(['startYear'])['numVotes'].transform('median')]

title_basics_ratings = title_basics_ratings[(~title_basics_ratings.runtimeMinutes.isna()) & \
                                            (~title_basics_ratings.averageRating.isna())  & \
                                            (~title_basics_ratings.numVotes.isna()) & 
                                            (~title_basics_ratings.genres.isna())]

title_basics_ratings[['startYear','runtimeMinutes']] = title_basics_ratings[['startYear','runtimeMinutes']].astype(int)
title_basics_ratings.averageRating = title_basics_ratings.averageRating.round(2)

In [ ]:
# prep principal crew per movie data
# TITLE PRINCIPALS
df_title_principals  = pickle.load(open(title_principals_file_prefix + ".ts.sav","rb"))
df_title_principals  = df_title_principals[df_title_principals.tconst.isin(title_basics_ratings.tconst)]

# add real names to nconsts
df_name_basics       = pickle.load(open("name.basics.ts.sav","rb"))[['nconst','primaryName']]
df_title_principals  = df_title_principals.merge(df_name_basics, on = 'nconst')

df_title_principals  = df_title_principals[['tconst','category','primaryName']]
nr_titles_per_name   = df_title_principals.groupby(['primaryName'])['tconst'].transform('count')
# decrease size of data by only looking at individuals with at least 5 titles associated with their names
df_title_principals  = df_title_principals[nr_titles_per_name >= 5]

In [ ]:
# label encode tconst (to integers) to ease burden on memory and computations based on data objects which 
# include tconst string values currently (but integer labels subsequently)
le_tconst                    = LabelEncoder()
le_tconst.fit(title_basics_ratings.tconst)
title_basics_ratings.tconst  = le_tconst.transform(title_basics_ratings.tconst)
df_title_principals.tconst   = le_tconst.transform(df_title_principals.tconst)

In [ ]:
# OBTAIN SIMILAR MOVIES PER MOVIE #
# per movie, obtain similar movies based on the length of the intersecting set of genres and people involved
# this should arguably result in quite similar results to the cosine similarity. 
# this processing takes a while to run as each of the around 140k movies is compared to all others, which corresponds to
# 140k * 140k or around 20bn operations. 
# To speed the computations up, the relevant apply function which iterates over all movies is supplemented with 
# swifter (based on Dask), which automatically parallellizes the apply function optimally across the running machine's cores.
# with my PCs specs (8 CPUs), this takes around 4 hours to run. 
# However, it is not necessary to rerun similar movies every update. Therefore, 'process_similar_movies' is set to False
# and the existing pickle file for similar movies will be loaded. 
# Set 'process_similar_movies' to True to rerun the similar movies algorithm. 
def find_similar_movies(tconst_input):
    selected_df     = df_similarity[df_similarity.tconst==tconst_input]
    df_comp         = selected_df[['keyword']]\
                 .merge(df_similarity[df_similarity.tconst!=tconst_input], on = 'keyword')
    intersects      = df_comp.groupby('tconst')['keyword'].count().sort_values(ascending = False)
    n_genres_movie  = df_n_genres.loc[df_n_genres.index==0,'n_genres'].item()
    similar_movies  = list(intersects[intersects>=n_genres_movie].head(10).index)
    return similar_movies

process_similar_movies = False
if process_similar_movies:
    
    # combine genres with names per movie as a base for similarity analysis
    df_tmp             = title_basics_ratings[['tconst','genres']]
    df_tmp['genre']    = df_tmp.genres.apply(lambda x: list(x.split(',')))
    df_tmp['n_genres'] = df_tmp.genre.apply(len)
    df_n_genres        = df_tmp[['tconst','n_genres']].set_index('tconst')
    genres_stacked     = df_tmp.drop(['genres','n_genres'], axis = 'columns').explode('genre')

    df_similarity = pd.concat((df_title_principals[['tconst','primaryName']].rename(columns = {'primaryName': 'keyword'}),\
                               genres_stacked.rename(columns = {'genre': 'keyword'})))\
                    .sort_values(by = 'tconst')
    
    mostVoted_movies = title_basics_ratings[title_basics_ratings.tconst.isin(df_title_principals.tconst.unique())]\
                      .sort_values(by = 'numVotes', ascending = False)\
                      .head(int(np.round(title_basics_ratings.shape[0])))\
                      .tconst
    mostVoted_movies.index = mostVoted_movies.values
    

    now             = datetime.now()
    similar_movies  = mostVoted_movies.swifter\
    .apply(lambda x: find_similar_movies(x))\
    .to_frame()\
    .reset_index()\
    .rename(columns = {'tconst' : 'similar_tconsts','index':'tconst'})

    later           = datetime.now()
    difference      = (later - now).total_seconds()
    print(f"creating list with similar movies took {difference/3600 :.2f} hours")

    with open('similar_movies.pkl', 'wb') as f:
        pickle.dump(similar_movies, f)  
else:
    similar_movies = pickle.load(open("similar_movies.pkl","rb"))

In [ ]:
# finalize and store data objects
title_basics_ratings = title_basics_ratings[title_basics_ratings.tconst.isin(df_title_principals.tconst.unique())]
title_basics_ratings = title_basics_ratings.merge(similar_movies, on = 'tconst', how = 'left')

# save sklearn label encoder for usage in the app
le_output = open('label encoder tconst.pkl', 'wb')
pickle.dump(le_tconst, le_output)
le_output.close()

# create table with genres one-hot encoded and with a label for each combination of genres that appears in the data
temp = title_basics_ratings.genres.dropna()
vec  = CountVectorizer(token_pattern='(?u)\\b[\\w-]+\\b', analyzer='word').fit(temp)
bag_of_genres =  vec.transform(temp)
unique_genres =  vec.get_feature_names()
genres = pd.DataFrame(bag_of_genres.todense(),\
                      columns=unique_genres,\
                      index=temp.index)
genres['genres'] = temp
genres = genres.drop_duplicates()
genres['genres_label'] = np.arange(genres.shape[0])
genres['genres_label'] = genres['genres_label'].astype(int)
genres                 = genres.reset_index(drop = True)

# add in genre label to df title_basics_ratings
title_basics_ratings = title_basics_ratings\
.merge(genres[['genres_label','genres']], on = 'genres', how = 'left')
title_basics_ratings.columns = ['tconst', 'Title', 'Year', 'Runtime', 'Genres',\
                                'Rating', 'Votes','similar_tconsts', 'genres_label']
with open('movies.pkl', 'wb') as f:
    pickle.dump(title_basics_ratings, f)

with open('genres.pkl', 'wb') as f:
    pickle.dump(genres, f)
    
df_title_principals_grouped = df_title_principals.groupby(['primaryName','category'])['tconst']\
                            .apply(lambda x: [x_single for x_single in x])\
                            .reset_index()\
                            .rename(columns = {'primaryName' :'Name','category':'Category'})

with open('people.pkl', 'wb') as f:
    pickle.dump(df_title_principals_grouped, f)
    
# store date of running to display in dashboard to notify users of the recency of the IMDB data they're viewing
with open('date_update.txt', 'w') as f:
    f.write(datetime.today().strftime('%Y-%m-%d'))

In [ ]:
# remove downloads from IMDB as they're no longer required and take up a lot of unnecessary storage
remove_files = ['name.basics.ts.sav',
                'name.basics.tsv.gz',
                'title.basics.tsv.gz',
                'title.basics.ts.sav',
                'title.principals.ts.sav',
                'title.principals.tsv.gz',
                'title.ratings.ts.sav',
                'title.ratings.tsv.gz']

for remove_file in remove_files:
    try:
        os.remove(remove_file)
    except OSError as e:
        print("Error: %s : %s" % (remove_file, e.strerror))